<h1> Part 1 : Data Ingestion & Storage </h1>


In [1]:
import polars as pl 
import duckdb
import requests
import os 
import streamlit as st
import plotly.express as px
import altair as alt

# The datasets are fetched from these URLs
url_tripData = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
url_lookupTable = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"

# The directory we are downloading and saving these files in.
destination_dir = "data/raw"

file_nameData = "yellow_tripdata_2024-01.parquet"
file_nameTable = "taxi_zone_lookup.csv"

# Making the directory
os.makedirs(destination_dir, exist_ok=True)


In [2]:
# Creating the full file path for both of these files
file_path1 = os.path.join(destination_dir, file_nameData)
file_path2 = os.path.join(destination_dir, file_nameTable)

response1 = requests.get(url_tripData)
response2 = requests.get(url_lookupTable)

# Saves the content file to the file paths in binary write 'wb' mode
with open(file_path1, 'wb') as file:
        file.write(response1.content)
with open(file_path2, 'wb') as file:
        file.write(response2.content)

# Printing status code
print(f'{response1.status_code}')
print(f'{response2.status_code}')

200
200


In [3]:
df1 = pl.read_parquet(url_tripData)

expected_columns = ["tpep_pickup_datetime", "tpep_dropoff_datetime",
"PULocationID", "DOLocationID" , "passenger_count", 
"trip_distance", "fare_amount", 
"tip_amount", "total_amount", 
"payment_type"]

missing_cols = set(expected_columns) - set(df1.columns)

# Validation check for if all the expected columns are in the dataset
if missing_cols:
    raise Exception(f"Validation Failed: Missing columns: {missing_cols}")

In [4]:
df1 = df1.with_columns([
    pl.col("tpep_pickup_datetime").cast(pl.Datetime, strict=False),
    pl.col("tpep_dropoff_datetime").cast(pl.Datetime, strict=False)
])

# Checking for null values
pickup_invalid = df1.select(
    pl.col("tpep_pickup_datetime").is_null().any()
).item()

dropoff_invalid = df1.select(
    pl.col("tpep_dropoff_datetime").is_null().any()
).item()

if pickup_invalid or dropoff_invalid:
    raise Exception("Validation Failed: Invalid datetime values detected.")

print(f'Number of rows: {len(df1):,}')
print(df1.select(expected_columns).describe())

Number of rows: 2,964,624
shape: (9, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statistic ┆ tpep_pick ┆ tpep_drop ┆ PULocatio ┆ … ┆ fare_amou ┆ tip_amoun ┆ total_amo ┆ payment_ │
│ ---       ┆ up_dateti ┆ off_datet ┆ nID       ┆   ┆ nt        ┆ t         ┆ unt       ┆ type     │
│ str       ┆ me        ┆ ime       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆ ---       ┆ ---       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
│           ┆ str       ┆ str       ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ count     ┆ 2964624   ┆ 2964624   ┆ 2.964624e ┆ … ┆ 2.964624e ┆ 2.964624e ┆ 2.964624e ┆ 2.964624 │
│           ┆           ┆           ┆ 6         ┆   ┆ 6         ┆ 6         ┆ 6         ┆ e6       │
│ null_coun ┆ 0         ┆ 0         ┆ 0.0       ┆ 

<h1> Part 2 : Data Transformation and Analysis </h1>

In [5]:
original_num = len(df1)

df_clean = df1.drop_nulls(subset=[
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "fare_amount"
])

df_clean = df_clean.filter((pl.col('trip_distance') > 0) & (pl.col('fare_amount') > 0) & (pl.col('fare_amount') < 500))

df_clean = df_clean.filter((pl.col('tpep_pickup_datetime')) < pl.col('tpep_dropoff_datetime'))

new_num = len(df_clean)

print(f'Number of rows cleaned : {original_num - new_num}')

print('\nThese rows were cleaned out because of the existance of null values in certain rows as well as improper pickup and dropoff times because the presence of these values can distort the data and give us a skewed and inaccurate analysis of the dataset.')

df_clean.head()

Number of rows cleaned : 95056

These rows were cleaned out because of the existance of null values in certain rows as well as improper pickup and dropoff times because the presence of these values can distort the data and give us a skewed and inaccurate analysis of the dataset.


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
i32,datetime[μs],datetime[μs],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,"""N""",186,79,2,17.7,1.0,0.5,0.0,0.0,1.0,22.7,2.5,0.0
1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.8,1,"""N""",140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.7,1,"""N""",236,79,1,23.3,3.5,0.5,3.0,0.0,1.0,31.3,2.5,0.0
1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.4,1,"""N""",79,211,1,10.0,3.5,0.5,2.0,0.0,1.0,17.0,2.5,0.0
1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.8,1,"""N""",211,148,1,7.9,3.5,0.5,3.2,0.0,1.0,16.1,2.5,0.0


In [6]:
derived_columns = df_clean.with_columns([((pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).dt.total_seconds() / 60).alias('trip_duration_minutes'),

                  (pl.col('trip_distance') / ((pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).dt.total_seconds() / 3600)).alias('trip_speed_mph'),

                  (pl.col('tpep_pickup_datetime').dt.hour()).alias('pickup_hour'),


            # Numbers 0-7 correspond to Monday to Sunday
                  (pl.col('tpep_pickup_datetime').dt.weekday()).alias('pickup_weekday')

                    ])

derived_columns.select(['trip_distance', 'trip_duration_minutes', 'trip_speed_mph',
'pickup_hour', 'pickup_weekday']).head()

print(f'Num of derived columns : {derived_columns}')


Num of derived columns : shape: (2_869_568, 23)
┌──────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ VendorID ┆ tpep_pick ┆ tpep_drop ┆ passenger ┆ … ┆ trip_dura ┆ trip_spee ┆ pickup_ho ┆ pickup_we │
│ ---      ┆ up_dateti ┆ off_datet ┆ _count    ┆   ┆ tion_minu ┆ d_mph     ┆ ur        ┆ ekday     │
│ i32      ┆ me        ┆ ime       ┆ ---       ┆   ┆ tes       ┆ ---       ┆ ---       ┆ ---       │
│          ┆ ---       ┆ ---       ┆ i64       ┆   ┆ ---       ┆ f64       ┆ i8        ┆ i8        │
│          ┆ datetime[ ┆ datetime[ ┆           ┆   ┆ f64       ┆           ┆           ┆           │
│          ┆ μs]       ┆ μs]       ┆           ┆   ┆           ┆           ┆           ┆           │
╞══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 2        ┆ 2024-01-0 ┆ 2024-01-0 ┆ 1         ┆ … ┆ 19.8      ┆ 5.212121  ┆ 0         ┆ 1         │
│          ┆ 1         ┆ 1         ┆       

In [7]:
conn = duckdb.connect()

conn.register("trips", derived_columns)

zones = pl.read_csv("data/raw/taxi_zone_lookup.csv")
conn.register("zones", zones)

<p> The following query answers question 6m : What are the top 10 busiest pickup zones by total number of trips?
   It answers this by joining the taxi lookup table to include zone names.
</p>

In [8]:
query1 = conn.execute('''
SELECT z.Zone,
       COUNT(*) AS trip_count
FROM trips t
JOIN zones z
ON t.PULocationID = z.LocationID
GROUP BY z.Zone
ORDER BY trip_count DESC
LIMIT 10''').fetchdf()

print(query1)

                           Zone  trip_count
0                Midtown Center      140141
1         Upper East Side South      140118
2                   JFK Airport      138426
3         Upper East Side North      133962
4                  Midtown East      104342
5     Times Sq/Theatre District      102958
6  Penn Station/Madison Sq West      102152
7           Lincoln Square East      101794
8             LaGuardia Airport       87693
9         Upper West Side South       86466


<p> The following query answers question 6n : What is the average fare amount for each hour of the day? 
    It is ordered from hour 0 to 23.</p>

In [9]:
query2 = conn.execute('''
SELECT pickup_hour,
       AVG(fare_amount) AS avg_fare
FROM trips
GROUP BY pickup_hour
ORDER BY pickup_hour''').fetchdf()

print(query2)

    pickup_hour   avg_fare
0             0  19.681288
1             1  17.735781
2             2  16.629330
3             3  18.536212
4             4  23.451590
5             5  27.500120
6             6  22.027144
7             7  18.753927
8             8  17.826515
9             9  17.947134
10           10  18.043603
11           11  17.628308
12           12  17.798799
13           13  18.421259
14           14  19.273210
15           15  19.114086
16           16  19.459183
17           17  18.120561
18           18  17.015240
19           19  17.629133
20           20  18.052625
21           21  18.292377
22           22  19.112203
23           23  20.246207


The following query answers question 6o : What percentage of trips use each payment type? It does this by computing the percentage distribution of trips across different payment types.

In [10]:
query3 = conn.execute('''SELECT payment_type,
       COUNT(*) * 100.0 /
       SUM(COUNT(*)) OVER() AS percentage
FROM trips
GROUP BY payment_type''').fetchdf()

print(query3)

   payment_type  percentage
0             0    4.014367
1             2   14.730824
2             1   80.093833
3             4    0.793011
4             3    0.367965


The following query answers question 6p : What is the average tip percentage (tip_amount/fare_amount) by day of week, for
credit card payments only?

In [11]:
query4 = conn.execute('''
SELECT pickup_weekday,
       AVG(tip_amount / fare_amount) * 100 AS avg_tip_percent
FROM trips
WHERE payment_type = 1
GROUP BY pickup_weekday''').fetchdf()

print(query4)

   pickup_weekday  avg_tip_percent
0               1        25.514116
1               2        25.730124
2               3        25.706625
3               4        29.734458
4               5        25.595701
5               6        26.293995
6               7        25.101118


The following query answers question 6q : What are the top 5 most common pickup-dropoff zone pairs?

In [12]:
query5 = conn.execute('''
SELECT pz.Zone AS pickup_zone,
       dz.Zone AS dropoff_zone,
       COUNT(*) AS trip_count
FROM trips t
JOIN zones pz
    ON t.PULocationID = pz.LocationID
JOIN zones dz
    ON t.DOLocationID = dz.LocationID
GROUP BY pickup_zone, dropoff_zone
ORDER BY trip_count DESC
LIMIT 5''').fetchdf()

print(query5)

             pickup_zone           dropoff_zone  trip_count
0  Upper East Side South  Upper East Side North       21641
1  Upper East Side North  Upper East Side South       19199
2  Upper East Side North  Upper East Side North       15193
3  Upper East Side South  Upper East Side South       14112
4         Midtown Center  Upper East Side South       10139


<h1> Part 3 : Visualization Dashboard </h1>

In [13]:
st.set_page_config( page_title='NYC Yellow Taxi Dashboard - January 2024', page_icon='taxi', layout='wide' )


2026-02-20 22:57:26.768 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [14]:
st.sidebar.header("Filters")

date_range = st.sidebar.date_input(
    "Select Date Range",
    value=[
        derived_columns.select(pl.col("tpep_pickup_datetime").min()).item().date(),
        derived_columns.select(pl.col("tpep_pickup_datetime").max()).item().date()
    ]
)

hour_range = st.sidebar.slider("Hour Range", 0, 23, (0, 23))

payment_types = derived_columns.select("payment_type").unique().to_series().to_list()

selected_payments = st.sidebar.multiselect(
    "Payment Type",
    payment_types,
    default=payment_types
)

2026-02-20 22:57:26.773 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:26.879 
  command:

    streamlit run /home/sham/my_projectfr/venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2026-02-20 22:57:26.880 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:26.881 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:26.884 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:26.885 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:26.886 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:26.887 Thread '

In [15]:
filtered_df = derived_columns.filter(
    (pl.col("pickup_hour") >= hour_range[0]) &
    (pl.col("pickup_hour") <= hour_range[1]) &
    (pl.col("payment_type").is_in(selected_payments))
)

filtered_pd = filtered_df.to_pandas()

if filtered_pd.empty:
    st.warning("No data available for selected filters.")
    st.stop()

col1, col2, col3, col4, col5 = st.columns(5)

col1.metric("Total Trips", len(filtered_pd))
col2.metric("Average Fare", f"${filtered_pd['fare_amount'].mean():.2f}")
col3.metric("Total Revenue", f"${filtered_pd['total_amount'].sum():,.2f}")
col4.metric("Avg Distance", f"{filtered_pd['trip_distance'].mean():.2f} mi")
col5.metric("Avg Duration", f"{filtered_pd['trip_duration_minutes'].mean():.2f} min")

2026-02-20 22:57:27.059 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:27.060 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:27.061 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:27.061 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:27.062 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:27.062 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:27.063 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-20 22:57:27.063 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

<h2> Visualization 1 : Bar Chart </h2>

In [16]:
zones = pl.read_csv("data/raw/taxi_zone_lookup.csv")

zones_pd = zones.to_pandas()

filtered_pd = filtered_pd.merge(
    zones_pd,
    left_on="PULocationID",
    right_on="LocationID",
    how="left"
)

top_zones = (
    filtered_pd.groupby("Zone")
    .size()
    .reset_index(name="trip_count")
    .sort_values("trip_count", ascending=False)
    .head(10)
)

fig1 = alt.Chart(top_zones).mark_bar().encode(
    y=alt.Y("Zone:O", sort='-x', title="Pickup Zone"),
    x=alt.X("trip_count:Q", title="Number of Trips"),
    tooltip=["Zone", "trip_count"]
).properties(
    title="Top 10 Busiest Pickup Zones",
    height=450
)

st.altair_chart(bar_chart, use_container_width=True)

st.markdown("""
Midtown Manhattan zones dominate pickup activity, reflecting commercial and tourism concentration.
""")

NameError: name 'bar_chart' is not defined

<h2> Visualization 2 : Line Chart</h2>

In [ ]:
avg_fare_hour = (
    filtered_pd.groupby("pickup_hour")["fare_amount"]
    .mean()
    .reset_index()
)

fig2 = px.line(avg_fare_hour,
               x="pickup_hour",
               y="fare_amount",
               title="Average Fare by Hour")

st.plotly_chart(fig2, width=True)

st.markdown("""
Fares peak during late evening and early morning hours.
Midday hours show more stable pricing.
""")

<h2> Visualization3 : Histogram </h2>

In [ ]:
filtered_small = filtered_pd[filtered_pd["trip_distance"] < 20]

fig3 = alt.Chart(filtered_small).mark_bar().encode(
    x=alt.X(
        "trip_distance:Q",
        bin=alt.Bin(maxbins=40),
        title="Trip Distance (miles)"
    ),
    y=alt.Y("count()", title="Number of Trips")
).properties(
    title="Trip Distance Distribution (0–20 miles)",
    height=450
)

st.altair_chart(histogram, use_container_width=True)

st.markdown("""
Most trips are short-distance (under 5 miles).
Long-distance trips are comparativelyrare.
""")

<h2> Visualization 4 : Pie Chart </h2> 

In [ ]:
payment_counts = (
    filtered_pd["payment_type"]
    .value_counts()
    .reset_index()
)

fig4 = px.pie(payment_counts,
              names="payment_type",
              values="count",
              title="Payment Type Breakdown")

st.plotly_chart(fig4, width=True)

st.markdown("""
Credit card payments dominate transactions.
Cash usage represents a smaller share of total trips.
""")

<h2>Visualization 5 : Heat Map </h2>

In [ ]:
heatmap_data = (
    filtered_pd.groupby(["pickup_weekday", "pickup_hour"])
    .size()
    .reset_index(name="trip_count")
)

fig5 = px.density_heatmap(
    heatmap_data,
    x="pickup_hour",
    y="pickup_weekday",
    z="trip_count",
    title="Trips by Day and Hour"
)

st.plotly_chart(fig5, width=True)

st.markdown("""
Weekday rush hours show peak demand during morning and evening commuting periods.
Weekend late-night activity is significantly higher than weekday nights.
""")

print(heatmap_data.head())

In [ ]:
tab1, tab2 = st.tabs(["Overview", "Temporal Analysis"])

<h4> AI Assistance was used for Part 1 - number 3 and Part 3 - number 10</h4>